In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [14]:
# Inputs the primary data into memory.
df = pd.read_csv("~/Dropbox/Coding/data-science-reu/node2vec/graph/email-Eu-core-department-labels.txt", sep=" ", header=None)

# After embedding with node2vec, we get a 1005 x 128 matrix and a label matrix of size 1 x 42.
# We can then reduce the rows in the embedded matrix to only include nodes with a label that is used more than 15 times.
node_ids = df[0]
label_ids = df[1]
label_occurrence_ids = df[1].value_counts().index
label_occurrence_frequencies = df[1].value_counts().values

# Finds the IDs of labels that are associated with highly connected nodes.
highly_connected_node_label_ids = [label_occurrence_ids[i]
                                   for i in range(len(label_occurrence_ids))
                                   if label_occurrence_frequencies[i] > 15]

# Finds the IDs of nodes & labels that are associated with high connectivity.
highly_connected_node_and_label_ids = {node_ids[i]: label_ids[i]
                                       for i in range(len(node_ids))
                                       if label_ids[i] in highly_connected_node_label_ids}

In [28]:
# Makes a dataframe out of an embedded data file.
X = pd.read_csv("~/Dropbox/Coding/data-science-reu/node2vec/emb/output.txt", sep=" ", header=None)
y = []

for node_id in X[0]:
    if node_id not in highly_connected_node_and_label_ids.keys():
        # Filters out rows with a label that has been used by less than 16 other rows.
        X = X[X[0] != node_id]
    else:
        # Otherwise, builds our y matrix of labels.
        y.append(highly_connected_node_and_label_ids[node_id])
        
# Prepares our training and testing data for the model.
kf = KFold(n_splits=10, shuffle=True)
X = np.array(X)
y = np.array(y)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  23  24  25  26  27  28  29  30  31  32  34  35  36  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 119 120 121 122 123 124 126 127 128 129 130 131 132
 133 134 135 136 137 138 139 140 142 143 144 145 146 147 148 149 150 152
 153 154 155 156 157 158 159 160 161 163 164 165 166 167 168 169 170 171
 174 175 177 180 181 182 183 184 185 186 187 188 189 190 191 193 194 196
 199 200 201 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265

In [29]:
# Instantiates, uses, and scores a logistic regression model from scikit-learn.
logisticRegr = LogisticRegression(solver="lbfgs", multi_class="auto")
logisticRegr.fit(X_train, y_train)
logisticRegr.predict(X_test)
logisticRegr.score(X_test, y_test)

/usr/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8674698795180723